In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb1
from xgboost import XGBClassifier

In [2]:
train_data=pd.read_csv("train_s3TEQDk.csv")
test_data=pd.read_csv("test_mSzZ8RL.csv")

In [3]:
X_train = train_data.drop(['ID','Is_Lead'], axis=1)
Y_train = train_data['Is_Lead'].values
X_test_data=test_data.drop(['ID'],axis=1)

In [4]:
X_train,X_test,y_train,y_test=train_test_split(X_train,Y_train,test_size = 0.2,random_state=42)

In [5]:
X_train['Credit_Product'].fillna('Missing',inplace=True)
X_test['Credit_Product'].fillna('Missing',inplace=True)
X_test_data['Credit_Product'].fillna('Missing',inplace=True)

In [6]:
#Doing Label Encoding to our Categeorical features for train_data
label_encoder=LabelEncoder()
X_train['Gender']=label_encoder.fit_transform(X_train['Gender'])
X_test['Gender']=label_encoder.fit_transform(X_test['Gender'])
X_test_data['Gender']=label_encoder.fit_transform(X_test_data['Gender'])

X_train['Region_Code']=label_encoder.fit_transform(X_train['Region_Code'])
X_test['Region_Code']=label_encoder.fit_transform(X_test['Region_Code'])
X_test_data['Region_Code']=label_encoder.fit_transform(X_test_data['Region_Code'])

X_train['Occupation']=label_encoder.fit_transform(X_train['Occupation'])
X_test['Occupation']=label_encoder.fit_transform(X_test['Occupation'])
X_test_data['Occupation']=label_encoder.fit_transform(X_test_data['Occupation'])

X_train['Channel_Code']=label_encoder.fit_transform(X_train['Channel_Code'])
X_test['Channel_Code']=label_encoder.fit_transform(X_test['Channel_Code'])
X_test_data['Channel_Code']=label_encoder.fit_transform(X_test_data['Channel_Code'])

X_train['Credit_Product']=label_encoder.fit_transform(X_train['Credit_Product'])
X_test['Credit_Product']=label_encoder.fit_transform(X_test['Credit_Product'])
X_test_data['Credit_Product']=label_encoder.fit_transform(X_test_data['Credit_Product'])

X_train['Is_Active']=label_encoder.fit_transform(X_train['Is_Active'])
X_test['Is_Active']=label_encoder.fit_transform(X_test['Is_Active'])
X_test_data['Is_Active']=label_encoder.fit_transform(X_test_data['Is_Active'])

In [7]:
scaling=MinMaxScaler()
X_train[['Age','Vintage','Avg_Account_Balance']]=scaling.fit_transform(X_train[['Age','Vintage','Avg_Account_Balance']])
X_test[['Age','Vintage','Avg_Account_Balance']]=scaling.fit_transform(X_test[['Age','Vintage','Avg_Account_Balance']])
X_test_data[['Age','Vintage','Avg_Account_Balance']]=scaling.fit_transform(X_test_data[['Age','Vintage','Avg_Account_Balance']])

#### a.LogisticRegression

In [ ]:
model1 = LogisticRegression()
model1.fit(X_train, y_train)
pred_prob1 = model1.predict_proba(X_test)
auc_score1 = roc_auc_score(y_test, pred_prob1[:,1])
print(auc_score1)

0.7256740410848981


In [ ]:
predictions_on_test_data=model1.predict(X_test_data)
submission=pd.read_csv("sample_submission_eyYijxG.csv")
submission['Is_Lead']=predictions_on_test_data
submission.to_csv('submissions_hackathon/LE_mms_logitic_regression_submission.csv',index=False)

#### b.Random Forest Classifier

In [ ]:
alpha=[10,50,100,500,1000,2000]
cv_log_error_array=[]
for i in alpha:
    model2=RandomForestClassifier(n_estimators=i,random_state=24,n_jobs=-1)
    model2.fit(X_train,y_train)
    pred_prob2 = model2.predict_proba(X_test)
    auc_score2 = roc_auc_score(y_test, pred_prob2[:,1])
    print(i)
    print("Roc_score=",auc_score2)

10
Roc_score= 0.8335547550267097
50
Roc_score= 0.8495813765275008
100
Roc_score= 0.8522954765343789
500
Roc_score= 0.8543893516289519
1000
Roc_score= 0.854936619895912
2000
Roc_score= 0.8550563587591994


In [ ]:
model2 = RandomForestClassifier(n_estimators=2000)
model2.fit(X_train, y_train)
pred_prob2 = model2.predict_proba(X_test)
auc_score2 = roc_auc_score(y_test, pred_prob2[:,1])
print(auc_score2)

0.8547065352958708


In [ ]:
predictions_on_test_data2=model2.predict(X_test_data)
submission=pd.read_csv("sample_submission_eyYijxG.csv")
submission['Is_Lead']=predictions_on_test_data2
submission.to_csv('submissions_hackathon/LE_mms_Random_Forest_submission.csv',index=False)

#### XGBoost

In [ ]:
model3 =XGBClassifier(subsample=1,n_estimators=1000,min_child_weight=7,max_depth=5,learning_rate=0.01,
                      colsample_bytree=1)
model3.fit(X_train,y_train)
pred_prob3 = model3.predict_proba(X_test)
auc_score3 = roc_auc_score(y_test, pred_prob3[:,1])
print(auc_score3)

0.8714234782309651


In [ ]:
predictions_on_test_data2=model3.predict(X_test_data)
submission=pd.read_csv("sample_submission_eyYijxG.csv")
submission['Is_Lead']=predictions_on_test_data2
submission.to_csv('submissions_hackathon/XGB_MMS_LE_submission.csv',index=False)

#### MLP

In [8]:
import tensorflow as tf 
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import PReLU,Dropout
from tensorflow.keras import Sequential
from keras.callbacks import Callback

In [9]:
model = Sequential()
model.add(Dense(400, input_dim = X_train.shape[1], kernel_initializer = 'he_normal'))
model.add(PReLU())
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(200, kernel_initializer='he_normal'))
model.add(PReLU())
model.add(BatchNormalization())    
model.add(Dropout(0.2))
model.add(Dense(50, kernel_initializer = 'he_normal'))
model.add(PReLU())
model.add(BatchNormalization())    
model.add(Dropout(0.2))
model.add(Dense(1, kernel_initializer = 'he_normal'))
#model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=[tf.keras.metrics.AUC(from_logits=True)])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 400)               4000      
_________________________________________________________________
p_re_lu (PReLU)              (None, 400)               400       
_________________________________________________________________
batch_normalization (BatchNo (None, 400)               1600      
_________________________________________________________________
dropout (Dropout)            (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               80200     
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 200)               200       
_________________________________________________________________
batch_normalization_1 (Batch (None, 200)               8

In [9]:
class LossHistory(Callback):
    def __init__(self, train, validation=None):
        super(LossHistory, self).__init__()
        self.validation = validation
        self.train = train
    def on_epoch_end(self, epoch, logs={}):
        X_train,Y_train = self.train[0], self.train[1]
        predictions = self.model.predict(X_train).ravel()
        logs['roc_auc_score_train'] = roc_auc_score(Y_train,predictions)
        if self.validation:
            X_val,Y_val = self.validation[0], self.validation[1]
            predictions = self.model.predict(X_val).ravel()
            logs['roc_auc_score_val'] = roc_auc_score(Y_val,predictions)
history_own=LossHistory(train=(X_train,y_train),validation=(X_test,y_test))

In [10]:
callbacks = [history_own]

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train,batch_size=8,epochs=20,callbacks=callbacks,validation_data=(X_test,y_test))

Epoch 1/20
24573/24573 [==============================] - 76s 3ms/step - loss: 3.1022 - accuracy: 0.7546 - val_loss: 2.2516 - val_accuracy: 0.8428 - roc_auc_score_train: 0.7081 - roc_auc_score_val: 0.7086
Epoch 2/20
24573/24573 [==============================] - 66s 3ms/step - loss: 1.3758 - accuracy: 0.7619 - val_loss: 0.6178 - val_accuracy: 0.8433 - roc_auc_score_train: 0.7821 - roc_auc_score_val: 0.7866
Epoch 3/20
24573/24573 [==============================] - 71s 3ms/step - loss: 0.5569 - accuracy: 0.7970 - val_loss: 0.4448 - val_accuracy: 0.8419 - roc_auc_score_train: 0.7924 - roc_auc_score_val: 0.7901
Epoch 4/20
24573/24573 [==============================] - 82s 3ms/step - loss: 0.4763 - accuracy: 0.8110 - val_loss: 0.4619 - val_accuracy: 0.8468 - roc_auc_score_train: 0.8361 - roc_auc_score_val: 0.8364
Epoch 5/20
24573/24573 [==============================] - 88s 4ms/step - loss: 0.4535 - accuracy: 0.8253 - val_loss: 0.5520 - val_accuracy: 0.8470 - roc_auc_score_train: 0.8563 - r

In [14]:
model.fit(X_train,y_train,batch_size=32,epochs=20,callbacks=callbacks,validation_data=(X_test,y_test))

Epoch 1/20
6144/6144 [==============================] - 27s 4ms/step - loss: 0.3929 - accuracy: 0.8410 - val_loss: 0.3627 - val_accuracy: 0.8529 - roc_auc_score_train: 0.8649 - roc_auc_score_val: 0.8624
Epoch 2/20
6144/6144 [==============================] - 37s 6ms/step - loss: 0.3878 - accuracy: 0.8447 - val_loss: 0.3634 - val_accuracy: 0.8533 - roc_auc_score_train: 0.8652 - roc_auc_score_val: 0.8620
Epoch 3/20
6144/6144 [==============================] - 28s 5ms/step - loss: 0.3926 - accuracy: 0.8442 - val_loss: 0.3640 - val_accuracy: 0.8526 - roc_auc_score_train: 0.8654 - roc_auc_score_val: 0.8621
Epoch 4/20
6144/6144 [==============================] - 27s 4ms/step - loss: 0.3892 - accuracy: 0.8454 - val_loss: 0.3643 - val_accuracy: 0.8546 - roc_auc_score_train: 0.8656 - roc_auc_score_val: 0.8625
Epoch 5/20
6144/6144 [==============================] - 26s 4ms/step - loss: 0.3916 - accuracy: 0.8407 - val_loss: 0.3666 - val_accuracy: 0.8544 - roc_auc_score_train: 0.8667 - roc_auc_sco

In [15]:
model.fit(X_train,y_train,batch_size=,epochs=20,callbacks=callbacks,validation_data=(X_test,y_test))

Epoch 1/20
12287/12287 [==============================] - 43s 4ms/step - loss: 0.3961 - accuracy: 0.8479 - val_loss: 0.3564 - val_accuracy: 0.8563 - roc_auc_score_train: 0.8699 - roc_auc_score_val: 0.8667
Epoch 2/20
12287/12287 [==============================] - 39s 3ms/step - loss: 0.3968 - accuracy: 0.8447 - val_loss: 0.3589 - val_accuracy: 0.8572 - roc_auc_score_train: 0.8710 - roc_auc_score_val: 0.8670
Epoch 3/20
12287/12287 [==============================] - 40s 3ms/step - loss: 0.3962 - accuracy: 0.8445 - val_loss: 0.3594 - val_accuracy: 0.8570 - roc_auc_score_train: 0.8701 - roc_auc_score_val: 0.8667
Epoch 4/20
12287/12287 [==============================] - 40s 3ms/step - loss: 0.4009 - accuracy: 0.8464 - val_loss: 0.3831 - val_accuracy: 0.8561 - roc_auc_score_train: 0.8692 - roc_auc_score_val: 0.8663
Epoch 5/20
12287/12287 [==============================] - 40s 3ms/step - loss: 0.4015 - accuracy: 0.8407 - val_loss: 0.3556 - val_accuracy: 0.8573 - roc_auc_score_train: 0.8702 - r

In [16]:
predictions_on_test_data3=model.predict(X_test_data)
submission=pd.read_csv("sample_submission_eyYijxG.csv")
submission['Is_Lead']=predictions_on_test_data3
submission.to_csv('submissions_hackathon/MLP3_callbacks2_MMS_LE_submission.csv',index=False)

#### LSTM

In [11]:
#Convert to numpy arrays
X_train, y_train = np.array(X_train), np.array(y_train)

#Reshape the data into 3-D array
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))

In [12]:
from keras import layers
model3 = Sequential()

model3.add(layers.LSTM(units = 50, return_sequences = True, input_shape = (9, 1)))
model3.add(layers.Dropout(0.2))

# Adding a second LSTM layer and Dropout layer
model3.add(layers.LSTM(units = 50, return_sequences = True))
model3.add(layers.Dropout(0.2))

# Adding a third LSTM layer and Dropout layer
model3.add(layers.LSTM(units = 50, return_sequences = True))
model3.add(Dropout(0.2))

# Adding a fourth LSTM layer and and Dropout layer
model3.add(layers.LSTM(units = 50))
model3.add(layers.Dropout(0.2))

# Adding the output layer
# For Full connection layer we use dense
# As the output is 1D so we use unit=1
model3.add(layers.Dense(units = 1,activation='sigmoid'))

In [13]:
model3.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 9, 50)             10400     
_________________________________________________________________
dropout (Dropout)            (None, 9, 50)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 9, 50)             20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 50)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 9, 50)             20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 9, 50)             0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                2

In [14]:
model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model3.fit(X_train,y_train,batch_size=8,epochs=20,callbacks=callbacks,validation_data=(X_test,y_test))

Epoch 1/20
24573/24573 [==============================] - ETA: 0s - loss: 0.4012 - accuracy: 0.8317

ValueError: in user code:

    c:\users\anuragreddy\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:941 test_function  *
        outputs = self.distribute_strategy.run(
    c:\users\anuragreddy\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\anuragreddy\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\anuragreddy\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\users\anuragreddy\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:909 test_step  **
    